In [ ]:
import pandas as pd
import geopandas as gpd
pd.set_option('display.max_colwidth', -1)

In [ ]:
! pip install geopandas geopy

In [ ]:
assmat_csv = 'data/assmat.csv'
df_assmat = pd.read_csv(assmat_csv, sep=',')
df_assmat.head(10)

In [ ]:
df_assmat.shape

In [ ]:
#cleanup
name = df_assmat['assmat'].str.split("Téléphone", n = 1, expand = True)
name[0] = name[0].str.strip()
name[1] = name[1].str.strip(' :')
name.head()
df_assmat['nom'] = name[0]
df_assmat['tel'] = name[1]
df_assmat.drop(columns=['assmat'], inplace=True)
df_assmat['adresse'] = df_assmat['adresse'].str.replace('\n',' ')
df_assmat.set_index('adresse', inplace=True)
df_assmat = df_assmat.drop("ADRESSE", axis=0)
df_assmat.reset_index(inplace=True)
df_assmat.head(10)


In [ ]:
# Import the geocoding tool
from geopandas.tools import geocode
from geopy.geocoders import Nominatim
nm_geolocator = Nominatim(user_agent="specify_your_app_name_here")

# Geocode addresses with Nominatim backend
#geo = geocode(df_assmat['adresse'].head(5), provider='nominatim', user_agent="assmat.0.0.1")
#geo = geocode(df_assmat['adresse'].head(10), provider='Nominatim',user_agent="assmat.0.0.1")
#geo = geocode(df_assmat['adresse'].head(100), user_agent="assmat.0.0.1")

#single_geo = geocode('6 rue bouvard 94120 fontenay sous bois', provider='nominatim') #works
#single_geo = geocode('AVENUE DE NEUILLY 94120 FONTENAY SOUS BOIS', provider='Nominatim') #notworks
#single_geo = geocode('AVENUE DE NEUILLY Porte n° 122 94120 FONTENAY SOUS BOIS', provider='Nominatim') #notworks
#single_geo = geocode('2 RESIDENCE DU CLOS D ORLEANS 94120 FONTENAY SOUS BOIS')#fixed in osm
#single_geo = geocode('AVENUE DE NEUILLY Porte n° 122 94120 FONTENAY SOUS BOIS') #works
single_geo = geocode('AVENUE DE NEUILLY Porte n° 122 94120 FONTENAY SOUS BOIS', provider='banfrance') #notworks

single_geo.head()
#geo.head()

In [ ]:
from geopy.geocoders import Nominatim, GeocodeFarm, BANFrance
#geolocator = Nominatim(user_agent="specify_your_app_name_here")
geolocator = BANFrance(user_agent="specify_your_app_name_here")

#geolocator = GeocodeFarm(user_agent="specify_your_app_name_here")# max 250 calls

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
df_assmat['location'] = df_assmat['adresse'].apply(geocode)

In [ ]:
df_assmat['point'] = df_assmat['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df_assmat['longitude'] = df_assmat['location'].apply(lambda loc: loc.longitude if loc else None)
df_assmat['latitude'] = df_assmat['location'].apply(lambda loc: loc.latitude if loc else None)
df_assmat.head()

In [ ]:
gdf = gpd.GeoDataFrame(
    df_assmat, geometry=gpd.points_from_xy(df_assmat.longitude, df_assmat.latitude))
gdf.drop(columns=['location', 'point'], inplace=True)
gdf.head()

In [ ]:
gdf.to_file('data/assmat_BanFrance.geojson', driver='GeoJSON')

In [ ]:
df = df_assmat[df_assmat.isna().any(axis=1)]
df.head()
#df.iloc[[0]]

